In [1]:
import pandas as pd

In [50]:
# 1. Read the data
column_names = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "salary"
]

df = pd.read_csv("../assets/adult.data.csv", names=column_names, na_values=' ?',
                 skipinitialspace=True)

df.drop(index=0, inplace=True)  # Drop the first row as it contains the column names

print(df.shape)

(32561, 15)


1. Скільки бакалаврів та магістрів (ознака education) представлено в цьому наборі даних?

In [52]:
bachelors_count = df[df['education'] == 'Bachelors'].shape[0]
masters_count   = df[df['education'] == 'Masters'].shape[0]

print("Bachelors:", bachelors_count)
print("Masters:", masters_count)

Bachelors: 5355
Masters: 1723


2. Який середній вік громадян Сполучених Штатів Америки (ознака native-country)?

In [53]:
df = df.astype({'age': 'int32'})

avg_age_us = df[df['native-country'] == 'United-States']['age'].mean()
print(avg_age_us)

38.65567363729859


3. Яка частка громадян Куби (ознака native-country)?

In [54]:
total_count = len(df)
cuba_count  = (df['native-country'] == 'Cuba').sum()
cuba_proportion = cuba_count / total_count

print("Number of Cuban nationals:", cuba_count)
print("Proportion of Cuban nationals:", cuba_proportion)

Number of Cuban nationals: 95
Proportion of Cuban nationals: 0.0029176008107859096


4. Які середні значення та середньоквадратичні відхилення віку для білої та чорної рас (ознака
race)?

In [56]:
race_stats = df.groupby('race')['age'].agg(['mean', 'std'])
white_mean = race_stats.loc['White', 'mean']
white_std  = race_stats.loc['White', 'std']
black_mean = race_stats.loc['Black', 'mean']
black_std  = race_stats.loc['Black', 'std']

print("White -- mean age:", white_mean,  "std:", white_std)
print("Black -- mean age:", black_mean,  "std:", black_std)

White -- mean age: 38.76988064423354 std: 13.782306359178051
Black -- mean age: 37.7679257362356 std: 12.759289573948983


5. Чи правда, що громадяни Ямайки працюють не менше 40 годин на тиждень (ознака hours-per-
week)?

In [57]:
df = df.astype({'hours-per-week': 'int32'})

jamaica_hours = df[df['native-country'] == 'Jamaica']['hours-per-week']
jamaica_min_hours = jamaica_hours.min()

if jamaica_min_hours >= 40:
    print("громадяни Ямайки працюють не менше 40 годин на тиждень")
else:
    print("громадяни Ямайки не працюють не менше 40 годин на тиждень")

громадяни Ямайки не працюють не менше 40 годин на тиждень


6. Виведіть статистику годин на тиждень для кожної раси (ознака race) та зарплати (ознака
salary). Використовуйте groupby та describe. Знайдіть мінімальну кількість годин, яку
працюють люди раси Asian-Pac-Islander з високою зарплатнею (більше 50К).

In [58]:
group_stats = df.groupby(['race','salary'])['hours-per-week'].describe()
print(group_stats)

                             count       mean        std   min   25%   50%  \
race               salary                                                    
Amer-Indian-Eskimo <=50K     275.0  39.410909  11.782482   3.0  40.0  40.0   
                   >50K       36.0  44.916667   9.854296  20.0  40.0  40.0   
Asian-Pac-Islander <=50K     763.0  38.425950  12.614856   1.0  36.0  40.0   
                   >50K      276.0  44.829710  11.134385   5.0  40.0  40.0   
Black              <=50K    2737.0  37.580197   9.957707   1.0  35.0  40.0   
                   >50K      387.0  44.382429  10.838834   2.0  40.0  40.0   
Other              <=50K     246.0  38.914634  11.173805   5.0  35.0  40.0   
                   >50K       25.0  44.920000   9.402659  30.0  40.0  40.0   
White              <=50K   20699.0  39.013624  12.597917   1.0  35.0  40.0   
                   >50K     7117.0  45.562035  11.026999   1.0  40.0  42.0   

                            75%   max  
race               sala

In [59]:
mask = (df['race'] == 'Asian-Pac-Islander') & (df['salary'] == '>50K')
asian_pac_high_salary_min = df.loc[mask, 'hours-per-week'].min()
print("(Asian-Pac-Islander, >50K):", asian_pac_high_salary_min)

(Asian-Pac-Islander, >50K): 5


7. Серед кого більша частка з високою зарплатнею (>50K): серед людей з вищою освітою чи ні?
Люди з вищою освітою – ті, що мають ознаку education однією з таких: Bachelors, Prof-school,
Assoc-acdm, Assoc-voc, Masters, Doctorate.

In [60]:
["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"]


['Bachelors', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', 'Masters', 'Doctorate']

In [61]:
higher_edu = ["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"]

df['higher_education'] = df['education'].isin(higher_edu)

high_edu_df   = df[df['higher_education']]
low_edu_df    = df[~df['higher_education']]

high_edu_rich = high_edu_df[high_edu_df['salary'] == '>50K'].shape[0] / high_edu_df.shape[0]
low_edu_rich  = low_edu_df[low_edu_df['salary'] == '>50K'].shape[0] / low_edu_df.shape[0]

print("Proportion of >50K among higher-educated:", high_edu_rich)
print("Proportion of >50K among lower-educated:", low_edu_rich)


Proportion of >50K among higher-educated: 0.4312476226702168
Proportion of >50K among lower-educated: 0.14996597867997277


8. Який мінімальний вік людей з цього набору даних? Скільки людей цього віку та який серед
них відсоток жінок, які заробляють більше 50К?

In [62]:
min_age = df['age'].min()
min_age_df = df[df['age'] == min_age]

count_min_age = min_age_df.shape[0]
count_min_age_female = (min_age_df['sex'] == 'Female').sum()

percent_female_min_age = 100.0 * count_min_age_female / count_min_age

# Among the min-age group who are women, how many earn >50K?
min_age_female_rich = min_age_df[
    (min_age_df['sex'] == 'Female') & (min_age_df['salary'] == '>50K')
].shape[0]

if count_min_age_female > 0:
    percent_female_rich = 100.0 * min_age_female_rich / count_min_age_female
else:
    percent_female_rich = 0  # or NaN, if there are no women in that group

print("Minimum age in dataset:", min_age)
print("Number of people with min age:", count_min_age)
print("Percentage of women among them: {:.2f}%".format(percent_female_min_age))
print("Percentage of those women who earn >50K: {:.2f}%".format(percent_female_rich))


Minimum age in dataset: 17
Number of people with min age: 395
Percentage of women among them: 47.09%
Percentage of those women who earn >50K: 0.00%


9. Порахуйте середньоквадратичне відхилення чоловіків та жінок для кожного значення ознаки
relationship.

In [63]:
std_by_relationship_sex = df.groupby(['relationship','sex'])['age'].std()
print(std_by_relationship_sex)

relationship    sex   
Husband         Female          NaN
                Male      12.024501
Not-in-family   Female    15.033271
                Male      12.546099
Other-relative  Female    16.091400
                Male      11.803060
Own-child       Female     8.375150
                Male       7.901183
Unmarried       Female    11.366622
                Male      12.297476
Wife            Female    11.231101
                Male       4.949747
Name: age, dtype: float64
